# Data Handling of Graphs

A graph is used to model pairwise relations (edges) between objects (nodes). A single graph in PyTorch Geometric is described by an instance of torch_geometric.data.Data, which holds the following attributes by default:

그래프는 pairwise 관계(edges)를 사용한다. single graph는 '''torch_geometric.data.Data''' 에 설명되어있다.

- data.x
    - Node feature matrix with shape [num_nodes, num_node_features]
    - [노드 갯수, 특성의 차원]

- data.edge_index
    - Graph connectivity in COO format with shape [2, num_edges] and type torch.long
    - 어떤 노드와 어떤 노드가 연결되어있는지 나타내는 특성인듯

- data.edge_attr
    - Edge feature matrix with shape [num_edges, num_edge_features]
    - 엣지의 피쳐

- data.y
    - Target to train against (may have arbitrary shape), e.g., node-level targets of shape [num_nodes, *] or graph-level targets of shape [1, *]
    - 학습시킬 것의 형태를 말하는 듯
    
- data.pos
    - Node position matrix with shape [num_nodes, num_dimensions]
    - 실제 위치를 나타내는듯?

None of these attributes is required. In fact, the Data object is not even restricted to these attributes. We can, e.g., extend it by data.face to save the connectivity of triangles from a 3D mesh in a tensor with shape [3, num_faces] and type torch.long.

이 특성들이 필요한 것은 아니다. 사실, Data 객체는 이 특성들에 제한되지않는다. 우리는 data.face를 통해서 [3, num_faces]의 크기를 가진 텐서의 3D mesh로부터 삼각형들의 연결성을 저장함으로써 그것을 확장시킬 수 있다(?)

We show a simple example of an unweighted and undirected graph with three nodes and four edges. Each node contains exactly one feature:

- unweighted
- undirected
- one feature

In [3]:
import torch
from torch_geometric.data import Data

edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float) #노드의 특성 여기선 1차원이니까 실수 하나


![graph](graph.svg)

In [5]:
data = Data(x=x, edge_index=edge_index)
data

Data(edge_index=[2, 4], x=[3, 1])

Note that '''edge_index''', i.e. the tensor defining the source and target nodes of all edges, is NOT a list of index tuples. If you want to write your indices this way, you should transpose and call '''contiguous''' on it before passing them to the data constructor:

In [6]:
import torch
from torch_geometric.data import Data

edge_index = torch.tensor([[0, 1],
                           [1, 0],
                           [1, 2],
                           [2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

In [8]:
data = Data(x=x, edge_index=edge_index.t().contiguous())
data #위와 같은 방법인데, edge_index를 다른 형태로 나타낸듯?

Data(edge_index=[2, 4], x=[3, 1])

Besides of being a plain old python object, '''torch_geometric.data.Data''' provides a number of utility functions, e.g.:

In [9]:
print(data.keys)

['x', 'edge_index']


In [10]:
#노드의 피쳐들을 나타냄.
print(data['x'])

tensor([[-1.],
        [ 0.],
        [ 1.]])


In [11]:
for key, item in data:
    print("{} found in data".format(key))

edge_index found in data
x found in data


In [12]:
'edge_attr' in data

False

In [13]:
data.num_nodes

3

In [14]:
#단방향을 하나로 치고, 2*2 = 4
data.num_edges

4

In [15]:
data.num_node_features

1

In [16]:
data.contains_isolated_nodes()

False

In [17]:
data.contains_self_loops()

False

In [19]:
#모두 양방향이니까
data.is_directed()

False

In [20]:
device = torch.device('cuda')
data = data.to(device)

You can find a complete list of all methods at '''torch_geometric.data.Data'''



# Common Benchmark Datasets 

PyTorch Geometric contains a large number of common benchmark datasets, e.g. all Planetoid datasets (Cora, Citeseer, Pubmed), all graph classification datasets from http://graphkernels.cs.tu-dortmund.de/ and their cleaned versions, the QM7 and QM9 dataset, and a handful of 3D mesh/point cloud datasets like FAUST, ModelNet10/40 and ShapeNet.

Initializing a dataset is straightforward. An initialization of a dataset will automatically download its raw files and process them to the previously described Data format. E.g., to load the ENZYMES dataset (consisting of 600 graphs within 6 classes), type:

- raw file을 받아서 프로세싱 하는 것은 Data format칸에서 설명하고 여기에서는 간단한 것만 해보겠다.

In [25]:
from torch_geometric.datasets import TUDataset

In [26]:
dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES')

Extracting /tmp/ENZYMES/ENZYMES/ENZYMES.zip
Processing...
Done!


In [28]:
print(len(dataset)) #600
print(dataset.num_classes) #6
print(dataset.num_node_features) #3

600
6
3


In [30]:
#그래프 600개중에 첫 번째 그래프
data = dataset[0]
data

Data(edge_index=[2, 168], x=[37, 3], y=[1])

In [31]:
data.is_undirected()

True

We can see that the first graph in the dataset contains 37 nodes, each one having 3 features. There are 168/2 = 84 undirected edges and the graph is assigned to exactly one class. In addition, the data object is holding exactly one graph-level target.

- 37개의 노드들이랑 3만큼의 피쳐를 가진다.
- 168/2 만큼의 undirected edge를 가지고 
- 그래프는 하나의 클래스를 가진다.
- 여기서 학습시킬 것은 그래프 level의 target이다.

We can even use slices, long or byte tensors to split the dataset. E.g., to create a 90/10 train/test split, type:

If you are unsure whether the dataset is already shuffled before you split, you can randomly permutate it by running:

- 섞고싶을때 이걸 써라
- 주석 친 두가지 모두 섞는 방법들임.

In [35]:
#dataset = dataset.shuffle() 

#perm = torch.randperm(len(dataset))
#dataset = dataset[perm]

train_dataset = dataset[:540] #ENZYMES(540)
test_dataset = dataset[540:] #ENZYMES(60)